In [305]:
import xml.etree.ElementTree as ET


def get_calibration(img_file):
    namespaces = {'leica': 'http://www.leica.com'}
    tree = ET.parse(img_file + '.anx')
    dx_basic_annotation_settings = tree.getroot()

    # Get image size metadata
    image_size_elm = dx_basic_annotation_settings.find('leica:ImageSize', namespaces)
    width = int(image_size_elm.find('leica:Width', namespaces).text)
    height = int(image_size_elm.find('leica:Height', namespaces).text)

    center_width = width / 2
    center_height = height / 2

    # Get image calibration metadata
    calibration = dx_basic_annotation_settings.find('leica:Calibration', namespaces)
    metres_per_pixel_elm = calibration.find('leica:MetresPerPixel', namespaces)
    metres_per_pixel = float(metres_per_pixel_elm.text)
    µm_per_pixel = metres_per_pixel * 1_000_000
    unit_per_pixel = metres_per_pixel * 10_000
    
    return {
        'metres_per_pixel': metres_per_pixel,
        'center_width': center_width,
        'center_height': center_height,
        'µm_per_pixel': µm_per_pixel,
        'unit_per_pixel': unit_per_pixel,
    }

In [306]:
# Get nematocyst locations


def get_points(img_file, calibration):
    tree = ET.parse(img_file + '.lmd')
    lm_data_store = tree.getroot()
    xml_points = lm_data_store.find('LMDataEntity').findall('LMMeasurementItemPoints')
    points = list(map(lambda point: {
        'raw_x_offset': float(point.find('PositionX').text),
        'raw_y_offset': float(point.find('PositionY').text),
        'index': int(point.find('PointIndex').text),
        'id': point.find('ItemId').text,
    }, xml_points))
    
    points = [p for p in points if p['index'] != 2]

    unit_per_pixel = calibration['unit_per_pixel']
    
    # arbitrary translation factors
    x_cal = 0
    y_cal = 0

    for point in points:
        point['x'] = (point['raw_x_offset'] * unit_per_pixel) + calibration['center_width'] + x_cal
        point['y'] = (point['raw_y_offset'] * unit_per_pixel) + calibration['center_height'] + y_cal
    
    return points

In [307]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg


def plt_image(img_flie, points):
    fig = plt.figure(figsize=(15, 15))
    img = mpimg.imread(img_file)
    plt.imshow(img)
    plt.plot([p['x'] for p in points], [p['y'] for p in points], 'ro')

In [309]:
import glob
files = glob.glob('/Users/paul/Documents/Streamline process/measured/**/*.tif')

for img_file in files:
    calibration = get_calibration(img_file)
    points = get_points(img_file, calibration)
    plt_image(img_file, points)